In [1]:
import pandas as pd
import numpy as np
import json
import math
import ast

In [2]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_leaflet as dl
import dash_leaflet.express as dlx
import plotly.express as px
from dash_extensions.javascript import Namespace

In [3]:
with open('astana_np3.json') as json_data:
    geojson = json.load(json_data)

In [4]:
df = pd.read_csv('temp_date.csv')

In [5]:
df['norm_temp'] = df['Т'].apply(lambda x : int(x))

In [6]:
df.dates = pd.to_datetime(df.dates)

In [7]:
df['onlydate'] = df.dates.apply(lambda x: x.date())

In [8]:
df_grouped = df.groupby(by='onlydate',as_index=False).agg({'Т':'mean'})

In [9]:
df_grouped

,onlydate,Т
0,2016-01-01,-18.6250
1,2016-01-02,-18.2875
2,2016-01-03,-17.3000
3,2016-01-04,-14.0625
4,2016-01-05,-15.2250
...,...,...
118,2016-04-28,7.5125
119,2016-04-29,7.7750
120,2016-04-30,9.6375
121,2016-05-01,12.0625


In [10]:
#http://127.0.0.1:8050/

In [11]:
print(px.colors.sequential.RdBu)

['rgb(103,0,31)', 'rgb(178,24,43)', 'rgb(214,96,77)', 'rgb(244,165,130)', 'rgb(253,219,199)', 'rgb(247,247,247)', 'rgb(209,229,240)', 'rgb(146,197,222)', 'rgb(67,147,195)', 'rgb(33,102,172)', 'rgb(5,48,97)']


In [12]:
px.colors.sequential.RdBu[5]

'rgb(247,247,247)'

In [13]:
def getColor(temper):
    if temper == abs(temper):
        for key in ctg:
            if temper > key:
                return ctg[key]
    else: 
        for key in reversed(ctg):
            if temper < key:
                return ctg[key]

In [14]:
classes = [25,20,15,10,5,0,-5,-10,-15,-20,-25]
colorscale = px.colors.sequential.RdBu
ctg = dict(zip(classes,colorscale))

In [15]:
ctg

{25: 'rgb(103,0,31)',
 20: 'rgb(178,24,43)',
 15: 'rgb(214,96,77)',
 10: 'rgb(244,165,130)',
 5: 'rgb(253,219,199)',
 0: 'rgb(247,247,247)',
 -5: 'rgb(209,229,240)',
 -10: 'rgb(146,197,222)',
 -15: 'rgb(67,147,195)',
 -20: 'rgb(33,102,172)',
 -25: 'rgb(5,48,97)'}

In [16]:
indices = list(range(len(classes) + 1))

In [17]:
df.iloc[[0]]['Т'][0]

-20.5

In [18]:
from sklearn import preprocessing

In [19]:
clustered_data = pd.read_csv('clustereddata.csv')
clustered_data.rename(columns={"delta t": "delta_t", "delta m": "delta_m"},inplace=True)
clustered_data['lon'] = clustered_data['lon_lat'].apply(lambda x: ast.literal_eval(x)[0])
clustered_data['lat'] = clustered_data['lon_lat'].apply(lambda x: ast.literal_eval(x)[1])
clustered_data.date = pd.to_datetime(clustered_data.date,format='%d.%m.%y')
clustered_data['onlydate'] = clustered_data.date.apply(lambda x: x.date())
scaler = preprocessing.StandardScaler().fit(clustered_data[['delta_t']])
X_scaled = scaler.transform(clustered_data[['delta_t']])
clustered_data['delta_t_scaled'] = X_scaled

In [20]:
clustered_data_mov_avg = pd.read_csv('clustereddata_mov_avg.csv')
clustered_data_mov_avg
clustered_data_mov_avg.rename(columns={"delta t": "delta_t", "delta m": "delta_m"},inplace=True)
clustered_data_mov_avg['lon'] = clustered_data_mov_avg['lon_lat'].apply(lambda x: ast.literal_eval(x)[0])
clustered_data_mov_avg['lat'] = clustered_data_mov_avg['lon_lat'].apply(lambda x: ast.literal_eval(x)[1])
clustered_data_mov_avg.date = pd.to_datetime(clustered_data_mov_avg.date,format='%d.%m.%y')
clustered_data_mov_avg['onlydate'] = clustered_data_mov_avg.date.apply(lambda x: x.date())
clustered_data_mov_avg

,Unnamed: 0,date,delta_t,delta_m,address,lon_lat,7pnt_mov_avg_dt,7pnt_mov_avg_dm,labels,lon,lat,onlydate
0,75,2016-03-16,20.82,1.180,"Шынтас 1, Нұр-Сұлтан","[71.39820238969105, 51.1981188]",21.040000,1.195714,1,71.398202,51.198119,2016-03-16
1,75,2016-03-16,16.10,8.700,"Сейфуллина 37, Нұр-Сұлтан","[71.42974036780862, 51.172306]",16.591429,8.928571,1,71.429740,51.172306,2016-03-16
2,75,2016-03-16,25.45,0.166,Манас 12 Нұр-Сұлтан,"[71.4893367, 51.1577124]",25.537143,0.162857,0,71.489337,51.157712,2016-03-16
3,75,2016-03-16,13.65,9.610,Букейхана 2,"[71.42645568909903, 51.15898305]",13.637143,9.812857,0,71.426456,51.158983,2016-03-16
4,74,2016-03-16,19.42,2.370,Букейхана 2/1,"[71.42697234398388, 51.15873]",19.490000,2.378571,0,71.426972,51.158730,2016-03-16
...,...,...,...,...,...,...,...,...,...,...,...,...
3387,10,2016-01-11,29.22,3.520,"Сейфуллина 5, Нұр-Сұлтан","[71.40179729812897, 51.1690605]",33.780000,4.602857,1,71.401797,51.169061,2016-01-11
3388,10,2016-01-11,27.68,2.950,"Туран 8, Нұр-Сұлтан","[71.47598661658415, 51.1398815]",37.415714,4.071429,0,71.475987,51.139882,2016-01-11
3389,10,2016-01-11,25.39,3.971,"Сейфуллина 25, Нұр-Сұлтан","[71.41874000720827, 51.1710548]",32.478571,4.996286,0,71.418740,51.171055,2016-01-11
3390,10,2016-01-11,32.84,6.619,"Женис 24, Нұр-Сұлтан","[71.4129873, 51.17078775]",41.111429,8.410286,3,71.412987,51.170788,2016-01-11


In [21]:
clustered_data.loc[(clustered_data['address'] == 'Шынтас 1, Нұр-Сұлтан') & (clustered_data['delta_t'] >= 30)].head(5)

,Unnamed: 0,index,date,delta_t,delta_m,address,lon_lat,labels,lon,lat,onlydate,delta_t_scaled
47,3,3,2016-01-04,30.95,1.780,"Шынтас 1, Нұр-Сұлтан","[71.39820238969105, 51.1981188]",1,71.398202,51.198119,2016-01-04,0.457175
55,3,3,2016-01-04,30.39,0.853,"Шынтас 1, Нұр-Сұлтан","[71.39820238969105, 51.1981188]",1,71.398202,51.198119,2016-01-04,0.397522
270,2,2,2016-01-03,34.45,1.960,"Шынтас 1, Нұр-Сұлтан","[71.39820238969105, 51.1981188]",1,71.398202,51.198119,2016-01-03,0.830004
278,2,2,2016-01-03,32.60,0.912,"Шынтас 1, Нұр-Сұлтан","[71.39820238969105, 51.1981188]",1,71.398202,51.198119,2016-01-03,0.632937
334,17,17,2016-01-18,34.66,1.990,"Шынтас 1, Нұр-Сұлтан","[71.39820238969105, 51.1981188]",1,71.398202,51.198119,2016-01-18,0.852373


In [22]:
clustered_data.loc[clustered_data['labels'] == 3].head(5)

,Unnamed: 0,index,date,delta_t,delta_m,address,lon_lat,labels,lon,lat,onlydate,delta_t_scaled
30,75,75,2016-03-16,25.74,6.010,"Женис 24, Нұр-Сұлтан","[71.4129873, 51.17078775]",3,71.412987,51.170788,2016-03-16,-0.097808
44,117,117,2016-04-27,14.76,0.260,"Сейфуллина 25, Нұр-Сұлтан","[71.41874000720827, 51.1710548]",3,71.418740,51.171055,2016-04-27,-1.267426
76,3,3,2016-01-04,37.28,9.003,"Женис 24, Нұр-Сұлтан","[71.4129873, 51.17078775]",3,71.412987,51.170788,2016-01-04,1.131463
108,59,59,2016-02-29,28.27,7.007,"Женис 24, Нұр-Сұлтан","[71.4129873, 51.17078775]",3,71.412987,51.170788,2016-02-29,0.171694
140,66,66,2016-03-07,27.31,6.480,"Женис 24, Нұр-Сұлтан","[71.4129873, 51.17078775]",3,71.412987,51.170788,2016-03-07,0.069432


In [23]:
scaler = preprocessing.StandardScaler().fit(clustered_data_mov_avg[['7pnt_mov_avg_dt']])
X_scaled = scaler.transform(clustered_data_mov_avg[['7pnt_mov_avg_dt']])
clustered_data_mov_avg['delta_t_scaled'] = X_scaled

In [24]:
def get_info():
    global count
    global df_grouped
    header = [html.H4("Ambient temperature")]
    return header + [html.B(df_grouped.iloc[[count]]['onlydate']), html.Br(),
                     "{:.2f} °C".format(df_grouped.iloc[[count]]['Т'][count])]

In [25]:
def get_data(label,count):
    global clustered_data_mov_avg
    current_date = df_grouped.iloc[[count]]['onlydate'][count]
    df_filtered = clustered_data_mov_avg[(clustered_data_mov_avg["labels"] == label) & (clustered_data_mov_avg["onlydate"] == current_date)]
    df_filtered = df_filtered[['lat', 'lon', 'address', 'date', '7pnt_mov_avg_dt', 'delta_t','delta_t_scaled']]  # drop irrelevant columns
    dicts = df_filtered.to_dict('rows')
    for item in dicts:
        item["tooltip"] = "{:.2f}".format(item["delta_t"])  # bind tooltip
        item["popup"] = 'Adress: {}<br>7-Point Moving Average:{}<br>Delta T: {}'.format(item["address"], 
                                    item['7pnt_mov_avg_dt'], item['delta_t'])  # bind popup
    geojson = dlx.dicts_to_geojson(dicts,lon="lon",lat="lat")  # convert to geojson
    geobuf = dlx.geojson_to_geobuf(geojson)  # convert to geobuf
    return geobuf

In [ ]:
count = 0
style = dict(weight=2, opacity=1, 
             fillColor=getColor(df_grouped.iloc[[count]]['Т'][count]),
             color = getColor(df_grouped.iloc[[count]]['Т'][count]),fillOpacity = 0.7)
chroma = "https://cdnjs.cloudflare.com/ajax/libs/chroma-js/2.1.0/chroma.min.js"
app = dash.Dash(external_scripts=[chroma], prevent_initial_callbacks=True)
next_count = 0
prev_count = 0
current_date = df_grouped.iloc[[count]]['onlydate'][count]
default_label = 0
labels = clustered_data["labels"].unique()
label_options = [dict(label=label, value=label) for label in labels]
dd_label = dcc.Dropdown(options=label_options, value=default_label, id="dd_label", clearable=False)

geojson_graph = dl.GeoJSON(data = geojson,
                    options=dict(style=style),
                    zoomToBounds=True,
                    zoomToBoundsOnClick=True,
                    hideout=dict(style=style),id = 'geojson')

minmax = dict(min=0, max=np.log(clustered_data['delta_t'].max()))

colorbar = dl.Colorbar(min=0, max=len(classes),classes=indices, 
                    colorscale=colorscale,
                    tickText=["<span style='display:block;text-align:left';>{}°C</span>".format(item) for item in classes],
                    tickValues=[item + 0.5 for item in indices[:-1]],position='bottomleft',
                    width=400, height=30,className = 'colorbar1')

next_button = html.Button('Next', id='submit-val-next',n_clicks=next_count,
                    style={"position": "relative", "bottom": "90px", "left": "10px", "z-index": "1000", "width": "200px"})

prev_button = html.Button('Prev', id='submit-val-prev',n_clicks=prev_count,
                    style={"position": "relative", "bottom": "90px", "left": "10px", "z-index": "1000", "width": "200px"})

info = html.Div(children=get_info(), id="info", className="info",
                style={"position": "absolute", "top": "10px", "right": "10px", "z-index": "1000"})

ns = Namespace("dlx", "scatter")
scatter_geojson = dl.GeoJSON(data=get_data(default_label,count), id="scatter_geojson", format="geobuf",
                     zoomToBoundsOnClick=True,  # when true, zooms to bounds of feature (e.g. cluster) on click
                     options=dict(pointToLayer=ns("pointToLayer"),style= {"radius":25,"weight":100,"fillOpacity":0.6}),  # how to draw points
                     hideout=dict(colorscale=px.colors.sequential.thermal,style={"z-index": "1000","display": "block","height":'100px'}
                                  ,colorProp = 'delta_t_scaled'))

app.layout = html.Div([dl.Map(children=[dl.TileLayer(),geojson_graph,colorbar,info,scatter_geojson],
                    style={'width': '98vw', 'height': '98vh'},
                    zoom=11, center=(51.1268, 71.4283)),
                    prev_button,next_button,html.Div(dd_label,
                    style={"position": "relative", "bottom": "150px",
                           "left": "10px", "z-index": "1000", "width": "200px"})])

def count_day(var1,var2):
    global next_count
    global prev_count
    global count
    result1 = (var1-next_count)
    result2 = (var2-prev_count)
    if result1:
        count +=1
        next_count +=1
    elif result2:
        count -= 1
        prev_count +=1
    return count

@app.callback([Output("geojson","hideout"),Output("geojson","options"),
               Output("info", "children"),Output("scatter_geojson","data")],
              [Input("submit-val-next",'n_clicks'),Input("submit-val-prev",'n_clicks'),Input("dd_label","value")])                    


def update_state(next_clicks,prev_clicks,label):
    count = count_day(next_clicks,prev_clicks)
    data = get_data(label,count)
    style = dict(weight=2, opacity=1, 
                 fillColor=getColor(df_grouped.iloc[[count]]['Т'][count]),
                 color = getColor(df_grouped.iloc[[count]]['Т'][count]),fillOpacity = 0.7)
    hideout = dict(style=style)
    options=dict(style=style)
    return hideout,options,get_info(),data
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Mar/2021 08:57:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2021 08:57:59] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2021 08:57:59] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2021 08:58:01] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2021 08:58:01] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2021 08:58:01] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2021 08:58:01] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2021 08:58:02] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2021 08:58:03] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2021 09:16:41] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2021 09:16:41] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2021 09:16:42] "POST /_dash-update-component

127.0.0.1 - - [13/Mar/2021 09:21:55] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2021 09:21:55] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2021 09:21:56] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2021 09:21:56] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2021 09:21:58] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2021 09:22:05] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2021 09:22:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2021 09:22:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2021 09:22:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2021 09:22:08] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2021 09:22:08] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2021 09:22:29] "POST /_dash-update-component HTTP/1.1" 200 -
127.